<h1>Training and Benchmarking a Model Using Model Zoo</h1>

This notebook provides a step-by-step guide on how to use the **STM32AI model zoo** to train, quantize, and benchmark an image classification model. The resulting model can be deployed on STM32 targets, making it ideal for edge computing applications. This notebook can be used in Jupyter Notebook environments, __(in local installation setup or in Google Colab)__ providing flexibility for users who prefer different development environments.

## License of the Jupyter Notebook

This software component is licensed by ST under BSD-3-Clause license, 
the "License";

You may not use this file except in compliance with the
License.

You may obtain a copy of the License at: https://opensource.org/licenses/BSD-3-Clause

Copyright (c) 2023 STMicroelectronics. All rights reserved.

<div style="border-bottom: 3px solid #273B5F">
<h2>Table of content</h2>
<ul style="list-style-type: none">
<li><a href="#install">1. Install necessary packages</a>
<li><a href="#config">2. Configure environment variables to access STM32Cube.AI Developer Cloud Services</a></li>
<li><a href="#upload">3. Upload the dataset</h2> </a></li>
<li><a href="#training">4. Training and Benchmarking the Model</a></li>
<li><a href="#results">5. Results</a></li>
  </ul>
</ul>
</div>

<div id="install">
    <h2>1. Install necessary packages</h2>
</div>

To get started, upload the model zoo package and clone the repository using the following command:

In [ ]:
# verifying if running in google colab
import sys
import os
if 'google.colab' in sys.modules:
  os.environ["GOOGLE_COLAB"]="1"
  print("Running the Notebook in Google Colab!")
else:
  os.environ["GOOGLE_COLAB"]="0"
  print("Running the Notebook local Jupyter Server!")

If running on Google Colab setting up the Python requirements for running modelzoo with `Python 3.10.x`.

In [ ]:
if os.environ["GOOGLE_COLAB"]=="1":
  !apt install python3.10 python3.10-distutils
  !curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py

  !python3.10 get-pip.py
  !python3.10 -m pip --version

  !apt install python3.10-venv

  !python3.10 -m pip install nvidia-cuda-runtime-cu11==11.8.89
  !python3.10 -m pip install nvidia-cudnn-cu11

  !wget https://developer.download.nvidia.com/compute/cuda/11.2.0/local_installers/cuda_11.2.0_460.27.04_linux.run
  !chmod +x cuda_11.2.0_460.27.04_linux.run
  !./cuda_11.2.0_460.27.04_linux.run --override --silent --toolkit

## (Optional to profit from GPU on Google Colab):
Go to : https://developer.nvidia.com/rdp/cudnn-archive and download cudnn v8.1.1 archive from nvidia website.

Selct _Download CuDNN v8.1.1 for CUDA 11.0, 11.1 and 11.2_.

Download `cuDNN Library for Linux (x86_64)`. You should have a file called: `cudnn-11.2-linux-x64-v8.1.1.33.tgz`. Then upload it under /content

In [ ]:
# in addition to this download and upload to the google colab the cudnn-11.2-linux-x64-v8.1.1.33.tgz file
if os.environ["GOOGLE_COLAB"]=="1":
  from google.colab import files

  # Upload "cudnn-11.2-linux-x64-v8.1.1.33.tgz" file 
  uploaded = files.upload()
  !tar -xvf cudnn-11.2-linux-x64-v8.1.1.33.tgz
  os.environ["LD_LIBRARY_PATH"] = (
  "/usr/local/cuda-11.2/targets/x86_64-linux/lib:"
  "/usr/local/lib/python3.11/dist-packages/nvidia/cufft/lib:"
  "/content/cuda/lib64:"
  + os.environ.get("LD_LIBRARY_PATH", "")
  )

### 1.1 Cloning the stm32ai-modelzoo-services repository

In [ ]:
!git clone https://github.com/STMicroelectronics/stm32ai-modelzoo-services.git

Or, you can upload a lighter version of STM32 model zoo by following these steps:
- On your local PC clone STM32AI model zoo git using the following command:
```
git clone https://github.com/STMicroelectronics/stm32ai-modelzoo-services.git
```
- Delete the .git directory.

- For image classification use-case, you can keep only the folders 'image_classification' and 'common', as well as the file 'requirements.txt', then delete the rest.

- Zip the repository as stm32ai-modelzoo-services.zip, and upload **stm32-modelzoo-services.zip** in your workspace.

- Then uncomment and run the cell below to unzip the folder.

In [ ]:
# import zipfile
# with zipfile.ZipFile('stm32ai-modelzoo-services.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

### 1.2 Installing the requirements 
Installing the requried python libraries and dependecies using `pip install` command.

In [ ]:
if os.environ["GOOGLE_COLAB"]=="1":
  !python3.10 -m venv st_zoo #create a virtual environment
  # install the requirements in the virtual environment
  !st_zoo/bin/pip install -r stm32ai-modelzoo-services/requirements.txt
  !st_zoo/bin/pip install matplotlib_inline IPython
else:
  !pip install -r stm32ai-modelzoo-services/requirements.txt


Next, run the following command to install the required packages

<div id="config">
    <h2>2. Configure environment variables to access STM32Cube.AI Developer Cloud Services</h2>
</div>
Set environment variables with your credentials to acces STM32Cube.AI Developer Cloud Services.

If you don't have an account yet go to: https://stedgeai-dc.st.com/home and click on sign in to create an account.

Then set the environment variables below with your credentials.


In [ ]:
import getpass
import os

os.environ['stmai_username'] = 'user.name@example.com'
print('Enter you password')
password = getpass.getpass()
os.environ['stmai_password'] = password

<div id="upload">
    <h2>3. Upload the dataset</h2>
</div>
The dataset can be uploaded as a zip archive named **dataset.zip** under the directory 'workspace/stm32ai-modelzoo-services/image_classification/datasets'.

The zip file shall contain a directory named "dataset" with one sub-directory per category, with images inside as below:

```bash
dataset_root_directory/
   class_a/
      a_image_1.jpg
      a_image_2.jpg
   class_b/
      b_image_1.jpg
      b_image_2.jpg
```
Other dataset formats are not supported. The only exceptions are the Cifar10/Cifar100 datasets. For these datasets, the official format in batches is supported.

The split between training and validation sets is done automatically by the scripts. However, it is also possible to upload specific training, validation, and test sets by defining specific paths in the user_config.yaml file.

In this tutorial we are going to use the flower dataset that can be downloaded directly from tensorflow repository: https://www.tensorflow.org/datasets/catalog/tf_flowers (Creative Commons By-Attribution License 2.0)

In [ ]:
dataset_name = 'tf_flowers' #@param ["custom", "tf_flowers"]
%cd stm32ai-modelzoo-services/image_classification

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import zipfile
import requests
import tarfile

def download_file(url, save_path):
    # Send a GET request to the URL
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        # Save the file in chunks
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File downloaded successfully: {save_path}")
        return True
    else:
        print(f"Failed to download file. HTTP Status Code: {response.status_code}")
        return False

def extract_tgz(file_path, extract_path="."):
    """
    Extracts a .tgz or .tar.gz file to the specified directory.
    
    Args:
        file_path (str): Path to the .tgz or .tar.gz file.
        extract_path (str): Directory where the contents should be extracted. Defaults to the current directory.
    """
    if tarfile.is_tarfile(file_path):
        with tarfile.open(file_path, "r:gz") as tar:
            tar.extractall(path=extract_path)
            print(f"Extracted '{file_path}' to '{extract_path}'")
    else:
        print(f"'{file_path}' is not a valid .tgz or .tar.gz file.")


# Define the path to the dataset directory
if dataset_name == 'tf_flowers':
    dataset_path = 'datasets/flower_photos'
    url = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    if download_file(url, dataset_path+'.tgz'):
        extract_tgz(dataset_path+'.tgz', extract_path='./datasets/')
else:
    path = 'datasets/dataset'
    with zipfile.ZipFile('datasets/dataset.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets')

# Get the list of class names
class_names = sorted([name for name in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, name))])
num_classes = len(class_names)
print(f"Classes: {class_names}")
print(f"Introducing samples from each class...")

# Print a photo from each class
fig, axs = plt.subplots(1, num_classes, figsize=(4*num_classes, 4))
for i, class_name in enumerate(class_names):
    class_path = os.path.join(dataset_path, class_name)
    image_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f)) and f.endswith('.jpg')]
    if len(image_files) == 0:
        img = np.zeros((224, 224, 3))
    else:
        img_path = os.path.join(class_path, random.choice(image_files))
        img = mpimg.imread(img_path)
    axs[i].imshow(img)
    axs[i].set_title(class_name)
    axs[i].axis('off')
plt.show()

<div id="training">
    <h2>4. Training and Benchmarking the Model</h2>
</div>

The STM32 model zoo is an invaluable resource that provides a wide range of use cases, including image classification, object detection, audio event detection, hand posture, and human activity recognition. The model zoo offers various services, including training, evaluation, prediction, deployment, quantization, benchmarking, and chained services. These services, such as chain_tbqeb, chain_tqe, chain_eqe, chain_qb, chain_eqeb, and chain_qd, are thoroughly explained in their respective readmes.

In this section, we will demonstrate how to train, quantize, evaluate, and benchmark a classification model using the chain_tbqeb service. We will use the MobileNet v2 0.35 model from the model zoo as an example, but you can also use your own custom model. To accomplish this, we will use the `user_config.yaml` file as a configuration file to specify the service and the set of configuration parameters, such as the model, dataset, number of epochs, and preprocessing parameters. Please feel free to review and adjust the training parameters as needed.

For a custom dataset, in the dataset section, modify:
*   the name and class_names accordingly.
*   training path: `training_path: ../datasets/dataset`

Then, you can tune the other parameters and save the file.

In [ ]:
if os.environ["GOOGLE_COLAB"]=='1':
  !../../st_zoo/bin/python stm32ai_main.py
else:
  %run stm32ai_main.py

<div id="results">
    <h2>5. Results</h2>
</div>
The trained and quantized models, along with any artifacts, plots, and figures related to the experiments, can be found in the 'workspace/stm32ai-modelzoo-services/image_classification/src/experiments_outputs' directory.

In [ ]:
import shutil
os.chdir('./src')
shutil.make_archive('experiments_outputs', 'zip', 'experiments_outputs')

# If running in Google colab following code will download the generated experiments_outputs folder.
if os.environ["GOOGLE_COLAB"]=="1":
  from google.colab import files

  # Download a specific file
  files.download('experiments_outputs.zip')  # Replace 'filename.ext' with the name of your file